In [2]:
import numpy as np
import pandas as pd
import pickle

In [4]:
with open('corpus/tag-based_attraction.list', 'rb') as f:
    corpus_attr = pickle.load(f)

In [5]:
corpus_attr

[['즐거운',
  '서울',
  '아침',
  '부터',
  '저녁',
  '까지',
  '정말',
  '알찬',
  '페스티벌',
  '입니다',
  '시청',
  '부터',
  '종로',
  '명동',
  '까지',
  '한',
  '번',
  '에',
  '즐길',
  '수',
  '있',
  '는',
  '코스',
  '이',
  '구요',
  '다양',
  '한',
  '부스',
  '특히',
  '음식',
  '이',
  '많',
  '아',
  '좋',
  '습니다',
  '당연히',
  '무료',
  '로',
  '진행',
  '되',
  '고',
  '서울시',
  '에서',
  '제공',
  '하',
  '는',
  '아리수',
  '도',
  '나눠',
  '줍니다',
  '직접',
  '참가',
  '하',
  '는',
  '공연',
  '도',
  '많',
  '아서',
  '아이',
  '들',
  '가족',
  '연인',
  '모두',
  '모두',
  '좋',
  '은',
  '페스티벌',
  '입니다'],
 ['한국',
  '에서',
  '제일',
  '큰',
  '불꽃',
  '축제',
  '한국',
  '에서',
  '제일',
  '큰',
  '불꽃',
  '축제',
  '입니다',
  '여러',
  '나라',
  '에서',
  '참가',
  '하',
  '기',
  '때문',
  '에',
  '정말',
  '구경',
  '하',
  '는',
  '사람',
  '이',
  '너무',
  '많',
  '아서',
  '지하철',
  '입구',
  '부터',
  '막힙니다',
  '저',
  '는',
  '주로',
  '근처',
  '한',
  '강다리',
  '에서',
  '보',
  '는데',
  '그게',
  '조금',
  '더',
  '여유',
  '롭',
  '게',
  '즐길',
  '수',
  '있',
  '습니다',
  '이',
  '역시',
  '사람',
  '이',
  '많',
  '지만',
  '그래도',
  

In [ ]:
top10 = model_attr.wv.most_similar(positive=['320359','324887'], negative=['1958940'])
for place in top10:
    print(df_places[df_places['placeId'] == int(place[0])].name)

In [ ]:
#유의미한 태그만 필터하는 패턴
#pattern = re.compile('MM|NNG|VA[+].*|VV[+].*|XR')